In [1]:
import copy
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def RMSE(img,img1):
    return np.sqrt(((img-img1)**2).mean())

In [3]:
# LZW_Encoder
def lzw_encoder(img,block,codesize):
    file=open("lzw.txt",'w')
    file.write("%s %s %s\n"%(img.shape[0],img.shape[1],block))
    i=0
    count=0
    
    while i<img.shape[0]:
        j=0
        while j<img.shape[1]:
            
            img1=np.zeros([block,block],dtype=np.uint8)
            img1[0:img.shape[0]-i,0:img.shape[1]-j]=img[i:min(i+8,img.shape[0]),j:min(j+8,img.shape[1])]
            
            dictionary={}
            temp=[]
            
            for m in range (0,256):
                temp=[m]
                
                dictionary[tuple(temp)]=m
                
            k=0
            l=0
            value=0
            temp=[]
            while(1):
                if k==block:
                    break
                temp.append(img1[k][l])
                
                if tuple(temp) in dictionary:                    
                    value=dictionary[tuple(temp)]
                else:
                    file.write("%s "%value)
                    count+=1
                    if len(dictionary)<(1<<codesize):
                        dictionary[tuple(temp)]=len(dictionary)
                        
                    
                    temp=[img1[k][l]]
                    value=dictionary[tuple(temp)]
                
                l+=1
                if l==block:
                    k+=1
                    l=0
            
            file.write("%s\n"%value)  
            count+=1
            j+=block
         
        i+=block
       
    original_size=8*img.shape[0]*img.shape[1]
    new_size=count*12
    compression=original_size/new_size
    
    file.close()     
    
    return compression
    
    

In [4]:
def lzw_decoder(codesize):
    file=open("lzw.txt",'r')
    line=file.readline()
    arr=line.split()
    height=int(arr[0])
    width=int(arr[1])
    block=int(arr[2])
    
    img=np.zeros([height,width],dtype=np.uint8)
    i=0
    j=0
    
    while(1):
        line=file.readline()
        if line=='':
            break
        arr=[int(s) for s in line.split()]
        
        img1=np.zeros(1,dtype=np.uint8)
        
        dictionary={}
        temp=[]
        
            
        for m in range (0,256):
            temp=[m]            
            dictionary[m]=temp
            
        
        
        temp=[]
        old=arr[0]
        img1[0]=arr[0]
        num=0
        c=arr[0]
        
        
        for k in range (1,len(arr)):            
            num=arr[k]
            if num in dictionary:                
                temp=dictionary[num].copy()
            else:                
                temp=dictionary[old].copy() 
                temp.append(c)
                
            
            img1=np.append(img1,np.array(temp))            
            c=temp[0]
            
            if len(dictionary)<(1<<codesize): 
                temp1=dictionary[old].copy()                
                temp1.append(c)                
                dictionary[len(dictionary)]=temp1                
             
            old=num
        
                    
        img1=img1.reshape(block,block)
        img[i:min(i+block,height),j:min(j+block,width)]=img1[0:min(i+block,height)-i,0:min(j+block,width)-j]
        j+=block
        if j>=width:
            i+=block
            j=0
        if i>=height:
            break
        
    file.close()
    return img  

In [ ]:


    
img=cv2.imread("checkerboard1024.tif",cv2.IMREAD_GRAYSCALE)
blocksize=8
codesize=12

compression_ratio=lzw_encoder(img,blocksize,codesize)

compressed_img=lzw_decoder(codesize)

error=RMSE(img,compressed_img)

hist, bins = np.histogram(img, 256, [0, 256])

# Probability density function
pdf = hist / float(np.sum(hist))
pdf = np.where(pdf != 0, pdf, np.finfo(float).eps)

# Calculate entropy
entropy = -np.sum(pdf * np.log2(pdf))

    

plt.figure(figsize=[18,5])
plt.subplot(141);plt.imshow(img,cmap="gray");plt.title("Original Image")
plt.subplot(142);plt.imshow(compressed_img,cmap="gray");plt.title("Compressed Image")


plt.figure(figsize=[20,4])
plt.subplot(143);plt.title("Analysis")
plt.axhline(y=entropy,color='r',linestyle='dashed',label="Entropy")
plt.axhline(y=compression_ratio,color='g',linestyle='-',label="Compression Ratio")
plt.legend(bbox_to_anchor=(0.25, 1.20))
plt.show()


